# RAG 기초

## Enviornment (.env)

In [ ]:
# .env 파일을 불러옵니다.
from dotenv import load_dotenv
load_dotenv()

## Tool 설정

In [ ]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b


tools = [add, multiply]

In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field

# Note that the docstrings here are crucial, as they will be passed along
# to the model along with the class name.
class Add(BaseModel):
    """Add two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class Multiply(BaseModel):
    """Multiply two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


tools = [Add, Multiply]

## Tool Calling / Function Calling


### 1) OpenAI

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
llm_with_tools = llm.bind_tools(tools=[add, multiply])

In [ ]:
query = "What is 3 * 12? Also, what is 11 + 49?"

response = llm_with_tools.invoke(query)
print(response)

In [ ]:
response.tool_calls

In [ ]:
llm_with_tools = llm.bind_tools(tools=[Add, Multiply])
response = llm_with_tools.invoke(query)
print(response)


In [ ]:
response.tool_calls

In [ ]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [HumanMessage(query)]
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))
messages

In [ ]:
llm_with_tools.invoke(messages)

### 2) Ollama

In [ ]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions

llm = OllamaFunctions(model="llama3", format="json")
llm_with_tools = llm.bind_tools(tools=[Add, Multiply])

# query = "What is 3 * 12? Also, what is 11 + 49?"
query = "what is 11 + 49? Also, What is 3 * 12?"
response = llm_with_tools.invoke(query)
print(response)

In [ ]:
response.tool_calls

In [ ]:
from langchain_core.messages import HumanMessage, FunctionMessage

# messages = [HumanMessage(query)]
messages = []
ai_msg = llm_with_tools.invoke(query)
messages.append(ai_msg)
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(FunctionMessage(tool_output, name=tool_call["name"]))
    
messages

In [ ]:
from langchain_community.llms import Ollama
Ollama(model="llama3").invoke(messages)

In [ ]:
messages

In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

chat_prompt = ChatPromptTemplate.from_template("""Breifely answer the question based on the following operation results:
                                               {context}
                                               
                                               Question:{question}
                                               Answer:""")
chat_model = ChatOllama(model="llama3")
llm_chain = chat_prompt | chat_model| StrOutputParser()
llm_chain.invoke({"context":messages, "question":query})

In [ ]:
def process_query(query):
    messages = []
    ai_msg = llm_with_tools.invoke(query)
    messages.append(ai_msg)
    for tool_call in ai_msg.tool_calls:
        selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
        tool_output = selected_tool.invoke(tool_call["args"])
        messages.append(FunctionMessage(tool_output, name=tool_call["name"]))   

    chat_prompt = ChatPromptTemplate.from_template("""다음 계산에 근거하여 질문에 간단히 한국어로 답하세요:
                                                {context}
                                                
                                                Question:{question}
                                                Answer:""")
    chat_model = ChatOllama(model="llama3")
    llm_chain = chat_prompt | chat_model| StrOutputParser()
    response = llm_chain.invoke({"context":messages, "question":query})
    if response:
        return response
    else:
        return "No response found."

query = "23 더하기 17은 얼마입니까?"
process_query(query)

## Structured Output

#### 1) (pydantic) Schema

In [ ]:
from typing import Optional
from langchain_core.pydantic_v1 import BaseModel, Field

class Car(BaseModel):
    """Information about a car."""
    make: Optional[str] = Field(default=None, description="The make of the car")
    model_name: Optional[str] = Field(default=None, description="The model name of the car")
    model_year: Optional[int] = Field(
        default=None, description="The year the car model was manufactured"
    )
    color: Optional[str] = Field(default=None, description="The color of the car")
    price: Optional[float] = Field(default=None, description="The price of the car")
    mileage: Optional[float] = Field(default=None, description="The mileage of the car")


#### 2) Prompt

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value.",
        ),
        ("human", "{text}"),
    ]
)

#### 3) OpenAI

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

runnable = prompt | llm.with_structured_output(schema=Car)

In [ ]:
text = """
현재 환경부와 산업통상자원부가 심사 중인 BYD의 소형 해치백 차량인 ‘돌핀’과 중형 세단 차량인 ‘씰’의 중국 내 최저 판매 가격은 각각 1900만 원, 3900만 원이다. 특히 돌핀은 국내에서 가장 값싼 경형 전기차인 ‘기아 레이EV(세제 혜택 전 2775만 원)’와 비교해도 압도적으로 저렴하다.
씰은 BYD의 셀투보디(CTB) 기술이 세계 최초로 적용된 차량으로 가격 대비 높은 성능을 자랑한다. CTB란 차량 본체와 배터리·배터리관리시스템(BMS) 등을 하나로 통합해 강성과 효율성을 모두 높이는 기술을 뜻한다. 두 차량 모두 유럽의 신차 안정성 프로그램(euro NCAP)에서 최고 등급을 받기도 했다.
한국 시장 진입을 위해 BYD가 현지 판매가와 유사한 수준으로 가격을 책정할 가능성도 있다. 통상 국내 시장 진입 시 가격을 더 높여잡는 게 일반적이지만 중국산 제품에 대한 한국의 부정적 인식을 고려해 가격 경쟁력을 최우선적으로 확보할 수 있다는 것이다. 스위스 투자은행(IB) UBS에 따르면 BYD는 배터리, 차량용 반도체, 소프트웨어 등 전체 부품 75%에 대한 수직 계열화를 이루면서 경쟁사 대비 30% 수준의 가격 우위를 확보하고 있다. 아울러 리튬·인산·철(LFP) 배터리에 대한 환경부의 불리한 규정에도 일정 수준의 보조금 확보도 가능하다. 현재 돌핀과 씰의 판매 가격은 국내 전기차 보조금 전액 지원 기준인 5500만 원을 충족한다. 유럽 인증 기준을 만족시키는 최대 427㎞(돌핀), 570㎞(씰)에 이르는 1회 충전 주행거리도 유리한 요소다.
BYD의 대항마로는 최근 기아가 출시한 소형 스포츠유틸리티차량(SUV) ‘EV3’가 꼽힌다. EV3는 니켈·코발트·망간(NCM) 배터리를 탑재해 롱레인지 모델 기준 1회 충전에 501㎞ 주행거리를 확보했다. 가격은 보조금 적용 시 3000만 원 중반대로 전기차 대중화라는 목표를 이루기 위한 기아의 주력 모델이다. KG모빌리티의 코란도EV(3000만 원대)도 BYD의 경쟁 상대다.
"""
response = runnable.invoke({"text": text})
print(response)

In [ ]:
from typing import List
class Data(BaseModel):
    """Extracted data about cars."""
    cars: List[Car] = Field(
        default=None, description="Extracted information about cars"
    )

#### 4) 여러 개의 Entity 추출

In [ ]:
from typing import List
class Data(BaseModel):
    """Extracted data about cars."""
    cars: List[Car] = Field(
        default=None, description="Extracted information about cars"
    )

runnable = prompt | llm.with_structured_output(schema=Data)
response = runnable.invoke({"text": text})
print(response)

### 5) Ollama

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """<|start_header_id|>system<|end_header_id|>
You are an expert extraction algorithm. Only extract relevant information from the text.
If you do not know the value of an attribute asked to extract, return null for the attribute's value.
<|eot_id|><|start_header_id|>user<|end_header_id|>

TEXT: {context}
QUESTION: {question}
JSON:
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""
)

llm = OllamaFunctions(model="llama3", format="json", temperature=0)
chain = prompt | llm.with_structured_output(schema=Car)
response = chain .invoke({"context": text, "question": "Describe 코란도"})
print(response)

## API 호출

In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field

class Stock(BaseModel):
    """Trading Stock"""

    ticker: Optional[str] = Field(default=None, description="""The ticker of the stock ("005930", "AAPL", ...)""")
    start_date: Optional[str] = Field(default=None, description="""The start trading date ("2021-01-01", ...)""")
    end_date: Optional[str] = Field(default=None, description="""The end trading date ("2021-12-31", ...)""")

class Market(BaseModel):
    """Stock market index"""

    ticker: Optional[str] = Field(default=None, description="""The ticker of the market index based on the following list(KS11, VIX, ...):
        - KS11: KOSPI 지수, 코스피 지수
        - KQ11: KOSDAQ 지수, 코스닥 지수
        - KS200: KOSPI 200, 코스피 200
        - DJI: 다우존스 지수, Dow Jones Industrial Average
        - IXIC: 나스닥 종합지수, NASDAQ Composite
        - S&P500: S&P500 지수, NYSE
        - RUT: 러셀2000 지수, Russell 2000
        - VIX: VIX 지수, CBOE Volatility Index
        - SSEC: 상해 종합지수, Shanghai Composite
        - HSI: 항셍지수, Hang Seng
        - N225: 일본 닛케이지수, Nikkei 225
        - FTSE: 영국 FTSE100, FTSE 100
        - FCHI: 프랑스 CAC40 지수, CAC 40
        - GDAXI: 독일 닥스지수, DAX 30""")
    start_date: Optional[str] = Field(default=None, description="""The start trading date ("2021-01-01", ...)""")
    end_date: Optional[str] = Field(default=None, description="""The end trading date ("2021-12-31", ...)""")

In [ ]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions

llm = OllamaFunctions(model="llama3", format="json")
llm_with_tools = llm.bind_tools(tools=[Stock, Market])

query = "삼성전자의 2023년 주가"

response = llm_with_tools.invoke(query)
print(response.tool_calls)

In [ ]:
query = "2024년 3월 테슬라"

response = llm_with_tools.invoke(query)
print(response.tool_calls)

In [ ]:
query = "2022년 나스닥 지수"

response = llm_with_tools.invoke(query)
print(response.tool_calls)

In [ ]:
query = "코스닥 지수 2019년 7월"

response = llm_with_tools.invoke(query)
print(response.tool_calls)

In [ ]:
### FinanceDataReader
import FinanceDataReader as fdr

df = fdr.DataReader("KQ11", "2019-07-01", "2019-07-31")
df.head()

In [ ]:
query = "2022년 3월 1일부터 2022년 3월 15일까지 삼성전자 주가"

ai_msg = llm_with_tools.invoke(query)
ticker = ai_msg.tool_calls[0]['args']['ticker']
start_date = ai_msg.tool_calls[0]['args']['start_date']
end_date = ai_msg.tool_calls[0]['args']['end_date']

print(ticker, start_date, end_date)
df = fdr.DataReader(ticker, start_date, end_date)
df.head
